# Algorithm 24: Compute All Atom Coordinates

This algorithm converts backbone frames and torsion angles to full atomic coordinates for all atoms in each residue.

## Algorithm Pseudocode

![computeAllAtomCoordinates](../imgs/algorithms/computeAllAtomCoordinates.png)

## Source Code Location
- **File**: `AF2-source-code/model/all_atom.py`
- **Functions**: `torsion_angles_to_frames`, `frames_and_literature_positions_to_atom14_pos`
- **Lines**: 400-500

## Process Overview

```
1. Backbone frames (from Structure Module)
        ↓
2. Torsion angles (χ1, χ2, χ3, χ4)
        ↓
3. Build sidechain frames using rotation matrices
        ↓
4. Place atoms using literature positions
        ↓
5. Full atom coordinates
```

In [ ]:
import numpy as np

np.random.seed(42)

In [ ]:
def make_rot_x(angle):
    """Algorithm 25: Rotation matrix around x-axis."""
    c, s = np.cos(angle), np.sin(angle)
    R = np.zeros(angle.shape + (3, 3))
    R[..., 0, 0] = 1
    R[..., 1, 1] = c
    R[..., 1, 2] = -s
    R[..., 2, 1] = s
    R[..., 2, 2] = c
    return R


def compute_all_atom_coordinates(backbone_frames, torsion_angles, aatype):
    """
    Compute All Atom Coordinates - Algorithm 24.
    
    Converts backbone frames and torsion angles to atom positions.
    
    Args:
        backbone_frames: (R, t) backbone rigid frames
            R: [N_res, 3, 3], t: [N_res, 3]
        torsion_angles: [N_res, 7, 2] sin/cos of torsion angles
            (omega, phi, psi, chi1, chi2, chi3, chi4)
        aatype: [N_res] amino acid types
    
    Returns:
        atom14_pos: [N_res, 14, 3] atom positions
    """
    N_res = len(aatype)
    R_bb, t_bb = backbone_frames
    
    print(f"Compute All Atom Coordinates")
    print(f"  Residues: {N_res}")
    print(f"  Backbone frames: R {R_bb.shape}, t {t_bb.shape}")
    print(f"  Torsion angles: {torsion_angles.shape}")
    
    # Step 1: Place backbone atoms (N, CA, C, O)
    # These are defined relative to the backbone frame
    # Literature positions for backbone atoms
    backbone_lit_pos = np.array([
        [0.0, 0.0, 0.0],      # N at origin
        [1.458, 0.0, 0.0],    # CA at ~1.458Å from N
        [2.009, 1.420, 0.0],  # C 
        [1.251, 2.498, 0.0],  # O
    ])
    
    # Transform backbone atoms to global frame
    atom14_pos = np.zeros((N_res, 14, 3))
    
    for i in range(4):  # N, CA, C, O
        # pos = R @ lit_pos + t
        atom14_pos[:, i] = np.einsum('nij,j->ni', R_bb, backbone_lit_pos[i]) + t_bb
    
    print(f"  Backbone atoms placed: N, CA, C, O")
    
    # Step 2: Build sidechain frames using torsion angles
    # Chi angles define sidechain conformations
    # For simplicity, we'll place CB (beta carbon) for non-glycine
    
    # CB position relative to CA (simplified)
    cb_lit_pos = np.array([-0.529, -0.774, 1.205])
    
    # Place CB for all residues (would be masked for GLY)
    atom14_pos[:, 4] = np.einsum('nij,j->ni', R_bb, cb_lit_pos) + t_bb
    
    print(f"  CB placed for sidechains")
    
    # Step 3: Build remaining sidechain atoms using chi angles
    # This is simplified - full implementation uses residue-specific geometry
    chi_angles = np.arctan2(torsion_angles[:, 3:, 0], torsion_angles[:, 3:, 1])
    
    for chi_idx in range(4):
        # Build rotation from chi angle
        R_chi = make_rot_x(chi_angles[:, chi_idx])
        
        # Simplified: place atoms along sidechain
        atom_idx = 5 + chi_idx
        if atom_idx < 14:
            offset = np.array([1.5 * (chi_idx + 1), 0.0, 0.0])
            local_pos = np.einsum('nij,j->ni', R_chi, offset)
            atom14_pos[:, atom_idx] = atom14_pos[:, 4] + local_pos
    
    print(f"  Sidechain atoms placed using chi angles")
    print(f"  Output atom14: {atom14_pos.shape}")
    
    return atom14_pos

In [ ]:
# Test
N_res = 32

# Initialize backbone frames
R_bb = np.tile(np.eye(3), (N_res, 1, 1))
t_bb = np.random.randn(N_res, 3) * 3  # Spread residues

# Random torsion angles (sin, cos pairs)
angles = np.random.randn(N_res, 7) * 0.5
torsion_angles = np.stack([np.sin(angles), np.cos(angles)], axis=-1)

# Random amino acid types
aatype = np.random.randint(0, 20, size=N_res)

print("Test Compute All Atom Coordinates")
print("="*50)

atom14_pos = compute_all_atom_coordinates((R_bb, t_bb), torsion_angles, aatype)

print(f"\nVerification:")
print(f"  CA-N distance (should be ~1.458Å): {np.linalg.norm(atom14_pos[0, 1] - atom14_pos[0, 0]):.3f}Å")

## Source Code Reference

```python
# From AF2-source-code/model/all_atom.py

def torsion_angles_to_frames(aatype, backb_to_global, torsion_angles_sin_cos):
  """Compute rigid group frames from torsion angles.
  
  Jumper et al. (2021) Suppl. Alg. 24 "computeAllAtomCoordinates"
  """
  # Build frames for each rigid group using torsion angles
  # ...

def frames_and_literature_positions_to_atom14_pos(aatype, all_frames):
  """Put atoms at literature positions in their frames."""
  # Look up literature atom positions for each residue type
  # Transform to global coordinates
  # ...
```